## Name: Limuelle Alamil
## Position Applying For: Software Developer

# Problem

Based on the data, prepare a program that will:
1. Determine the client who has the highest loan balance
2. Come up with a report summarizing the total balance and total savings balance per client. 
Output report must be in excel format.

# Solution

## Importing libraries

The Pandas library was used in this program. Pandas is a software library written for the Python programming language for data manipulation and analysis. In particular, it offers data structures and operations for manipulating numerical tables and time series.

In [1]:
import pandas as pd

## Data reading and pre-processing

I transferred the given data to a CSV file. The 'ACCT_MASTER' table was transferred to the `Account_Master.csv` file. Let's call this the `accounts`. The 'ID_MASTER' table was transferred to the `Id_Master.csv` file. Let's call this the `clients`.

Then, I sorted the tables based on their ID (IDNo) to make it easier to read.

#### Accounts

In [2]:
accounts = pd.read_csv("Account_Master.csv")
sorted_accts = accounts.sort_values(by=['IDNo', 'AccTyp'])
sorted_accts.reset_index(drop=True, inplace=True)
sorted_accts

,IDNo,AccTyp,AcctNo,ChkDgt,Balance
0,1,LN,1111,25,130212.55
1,1,LN,4444,2,98000.12
2,1,LN,6666,3,1500894.00
3,1,SA,7777,5,45222.12
4,1,SA,4321,89,11960.89
5,2,LN,8888,6,32341.51
6,2,SA,2222,26,212896.75
7,2,SA,5555,99,1450000.11
8,2,SA,1222,21,4345.67
9,3,SA,3333,1,3561.56


#### Clients

In [3]:
clients = pd.read_csv("Id_Master.csv")
clients

,IDNo,ClntName
0,1,"Pascual, Piolo"
1,2,"Rosales, Jericho"
2,3,"Dantes, Dingdong"
3,4,"Padilla, Daniel"
4,5,"Rodriguez, Tom"
5,10,"Santos, Eric"


## Computing the answer

Three dictionaries were initialized. They will contain the total loan balance, total savings balance, and total balance (savings - loans) of each client, respectively. Specifically, the client's ID will serve as the key and the client's balance will serve as the value (e.g., { [id]:[balance] } )

In [4]:
total_loan = {}
total_sav = {}
total_bal = {}

A double `for` loop is used to solve the problem. The first loop will iterate through the list of clients. Then, each client's ID will be cross-checked against the client IDs in the accounts table using another `for` loop. **This is how the account data of the client (e.g., loan balance and savings balance) will be accessed.**

Then, each row in the `accounts` is checked whether it is a loan or savings account. If the row is a loan account (LN), the loan balance will be added to the client's value in the `total_loan` dictionary and will be deducted from the client's value in the `total_bal` dictionary, otherwise (SA), it will be added to the client's value in the `total_sav` dictionary and will be added to the client's value in the `total_bal` dictionary. The owner of the account can be determined using the ID. **This is how the total loan balance, total savings balance, and total balance of each client is computed and temporarily stored in the dictionaries.**

Once all the rows in the accounts table are checked, the three dictionaries will be converted into a list so that it can be appended as a column to the dataframe (table). **This is how the totals are displayed.**

In [5]:
# will loop through each client in the clients table
for c in range(0, clients[clients.columns[0]].count()):

    # will retrieve the current client's ID
    c_id = clients.iloc[c]['IDNo']

    # will initialize the total loan, total savings, and total balance of the current client to 0; this will increment or decrement later
    total_loan[c_id] = 0
    total_sav[c_id] = 0
    total_bal[c_id] = 0

    # will loop through each account in the accounts table
    for a in range(0, sorted_accts[sorted_accts.columns[0]].count()):

        # will retrieve the current account's client ID
        a_id = sorted_accts.iloc[a]['IDNo']
        # will retrieve the current account's type (LN or SA)
        a_type = sorted_accts.iloc[a]['AccTyp']

        # will check if the current account's client ID is the current client in the loop; if not, exit the iteration and proceed to next account
        if c_id == a_id:
            # if the current account is a loan
            if a_type == 'LN':
                a_loan_bal = sorted_accts.iloc[a]['Balance']
                # add the loan balance to the client's total loan
                total_loan[c_id] += a_loan_bal
                # subtract the loan balance to the client's total balance
                total_bal[c_id] -= a_loan_bal
            # if the current account is a savings
            else:
                a_sav_bal = sorted_accts.iloc[a]['Balance']
                # add the savings balance to the client's total savings
                total_sav[c_id] += a_sav_bal
                # add the savings balance to the client's total balance
                total_bal[c_id] += a_sav_bal

# converts the dictionaries into lists so they can be displayed in the dataframe (table)
total_loan_list = list(total_loan.values())
total_sav_list = list(total_sav.values())
total_bal_list = list(total_bal.values())

# displays the list in the dataframe (table)
clients['Total Loan Balance'] = total_loan_list
clients['Total Savings Balance'] = total_sav_list
clients['Total Balance'] = total_bal_list

# Answers

#### Report summarizing the total balance and total savings balance per client

In [6]:
clients.to_csv('Account_Summary_Report.csv', index=False)
clients

,IDNo,ClntName,Total Loan Balance,Total Savings Balance,Total Balance
0,1,"Pascual, Piolo",1729106.67,57183.01,-1671923.66
1,2,"Rosales, Jericho",32341.51,1667242.53,1634901.02
2,3,"Dantes, Dingdong",0.00,47112.71,47112.71
3,4,"Padilla, Daniel",653342.18,562254.18,-91088.00
4,5,"Rodriguez, Tom",0.00,90431.12,90431.12
5,10,"Santos, Eric",0.00,145.78,145.78


The summary report named `Account_Summary_Report` is now exported as a CSV file in the root folder.

#### The client who has the highest loan balance

In [7]:
# will check what row has the highest value in the Total Loan Balance column
highest_loan_index = clients['Total Loan Balance'].idxmax()
# will retrieve the row's client name (e.g., 'Pascual, Piolo')
highest_loan_client = clients.iloc[highest_loan_index]['ClntName']
highest_loan_client

'Pascual, Piolo'